In [110]:
import json

colombia_geojson = json.load(open("/Users/kak4007/Documents/hackathons/cornell_health_2022/HealthHack2022/colombia.geo.json"))
colombia_geojson["features"][0]
colombia_geojson["features"][-1]


{'type': 'Feature',
 'properties': {'DPTO': '88',
  'NOMBRE_DPT': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
  'AREA': 49602521.129,
  'PERIMETER': 71946.251,
  'HECTARES': 4960.252},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-81.7129569648294, 12.594562811518577],
     [-81.71283785603457, 12.594715835636555],
     [-81.71271131159209, 12.594885886973241],
     [-81.71260367437974, 12.595007168710904],
     [-81.71248344500896, 12.595129795489784],
     [-81.71241260846307, 12.59520875639679],
     [-81.71228513486054, 12.595311231005786],
     [-81.71215025198923, 12.595400315583289],
     [-81.71202941457501, 12.595489156417274],
     [-81.71192724945676, 12.595579926337129],
     [-81.71172632131714, 12.595743382337233],
     [-81.71161240664665, 12.595818584393111],
     [-81.71150306106863, 12.59587680892798],
     [-81.71137155653828, 12.595962455035778],
     [-81.71122257279244, 12.596016860994911],
     [-81.71111806571393, 12.596041205025173]

In [80]:
import pandas as pd

dpts = ['amazonas', 'antioquia']

precips = pd.DataFrame()

for dpt in dpts:
    precip = pd.read_csv(f"colombia_{dpt}_precip.csv", skiprows=3)
    precip['NOMBRE_DPT'] = dpt.upper()
    precip = precip.rename(columns={'Unnamed: 0': 'year'})
    precips = pd.concat([precips, precip])
precips

# precips.to_csv("precips_example.csv")

,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,NOMBRE_DPT
0,1901,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
1,1902,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
2,1903,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
3,1904,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
4,1905,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2016,136.53,129.38,194.03,198.58,396.10,306.57,152.96,372.74,282.54,319.36,287.16,190.67,ANTIOQUIA
116,2017,166.67,89.25,133.92,251.69,392.25,260.12,190.87,227.94,305.26,340.76,322.34,85.93,ANTIOQUIA
117,2018,86.30,95.46,181.34,211.68,392.52,216.75,193.21,173.51,290.55,293.48,285.10,167.14,ANTIOQUIA
118,2019,87.73,95.53,113.55,135.48,217.35,252.90,345.61,297.80,311.83,221.51,349.35,220.91,ANTIOQUIA


In [92]:
min_precip = precips.drop(columns=['year','NOMBRE_DPT']).dropna().to_numpy().min()
max_precip = precips.drop(columns=['year','NOMBRE_DPT']).dropna().to_numpy().max()
print(min_precip, max_precip)


23.43 709.09


In [26]:
jul_precips = precips[['Jul','NOMBRE_DPT']].groupby('NOMBRE_DPT').mean()
jul_precips.reset_index(inplace=True)
jul_precips

,NOMBRE_DPT,Jul
0,AMAZONAS,245.456218
1,ANTIOQUIA,225.933109


In [40]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()
candidates = df.winner.unique()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.P("Candidate:"),
    dcc.RadioItems(
        id='candidate', 
        options=[{'value': x, 'label': x} 
                 for x in candidates],
        value=candidates[0],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="choropleth"),
])

@app.callback(
    Output("choropleth", "figure"), 
    [Input("candidate", "value")])
def display_choropleth(candidate):
    fig = px.choropleth(
        jul_precips, geojson=colombia_geojson, color="Jul",
        locations='NOMBRE_DPT', featureidkey="properties.NOMBRE_DPT",
        projection="mercator", range_color=[0, 6500])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

app.run_server(debug=True)



Dash app running on http://127.0.0.1:8050/


In [127]:
weather_dpts = []
for ft in colombia_geojson["features"]:
    weather_dpts.append(ft["properties"]["NOMBRE_DPT"])
weather_dpts.sort()
print(weather_dpts)

cases = pd.read_csv("case.summary.noclimate.txt")
cases['admin_ascii'] = cases['admin_ascii'].str.upper()
cases_dpts = cases['admin_ascii'].unique()
cases_dpts.sort()
# print(cases_dpts)

# change case data dpt names to match geojson:
cases = cases.replace({"SAN ANDRES Y PROVIDENCIA": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
                       "NARINO": "NARIÑO",
                       "BOGOTA": "SANTAFE DE BOGOTA D.C"})
cases_dpts = cases['admin_ascii'].unique()
cases_dpts.sort()
print(cases_dpts)


['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ATLANTICO', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SANTAFE DE BOGOTA D.C', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA']
['AMAZONAS' 'ANTIOQUIA' 'ARAUCA'
 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA' 'ATLANTICO'
 'BOLIVAR' 'BOYACA' 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO'
 'CORDOBA' 'CUNDINAMARCA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'MAGDALENA'
 'META' 'NARIÑO' 'NORTE DE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA'
 'SANTAFE DE BOGOTA D.C' 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA'
 'VAUPES' 'VICHADA']


In [130]:
cases.head()
print(cases['year'].max())
cases[(cases['month'] == 7.0) & (cases['year'] == 2017)]

2017


,month,city_ascii,year,total_cases,confirmed_cases,city_ascii.1,lat,lng,admin_ascii,population,population_proper
123,7.0,abrego,2017,4,2,abrego,8.0806,-73.2205,NORTE DE SANTANDER,NaN,NaN
268,7.0,acacias,2017,19,1,acacias,3.9889,-73.7647,META,75252.0,75252.0
289,7.0,acandi,2017,1,1,acandi,8.5114,-77.2778,CHOCO,NaN,NaN
394,7.0,acevedo,2017,1,0,acevedo,1.8047,-75.8897,HUILA,39877.0,39877.0
473,7.0,achi,2017,1,0,achi,8.5692,-74.5561,BOLIVAR,23051.0,23051.0
...,...,...,...,...,...,...,...,...,...,...,...
49424,7.0,yarumal,2017,1,0,yarumal,6.9633,-75.4172,ANTIOQUIA,48556.0,48556.0
49509,7.0,yolombo,2017,1,1,yolombo,6.5978,-75.0122,ANTIOQUIA,25647.0,25647.0
49636,7.0,yopal,2017,32,1,yopal,5.3306,-72.3906,CASANARE,179355.0,179355.0
49816,7.0,yumbo,2017,4,2,yumbo,3.5778,-76.4944,VALLE DEL CAUCA,92192.0,92192.0


gray out "GUAINIA"?

In [96]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# fig = px.choropleth(
#     jul_precips, geojson=colombia_geojson, color="Jul",
#     locations='NOMBRE_DPT', featureidkey="properties.NOMBRE_DPT",
#    projection="mercator", range_color=[0, 6500])
# fig.update_geos(fitbounds="locations", visible=True)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(figure=fig)
])
app.run_server(debug=True)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)

fig = px.choropleth_mapbox(jul_precips, geojson=colombia_geojson, locations='NOMBRE_DPT', color='Jul',
                           color_continuous_scale="Viridis",
                           range_color=(min_precip, max_precip),
                           mapbox_style="carto-positron",
                           zoom=3,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": 74.2973328}
                          )


Dash app running on http://127.0.0.1:8050/


In [53]:
import(go)

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="Colombia Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body',
        children=[


            dcc.Graph(
                id='map-plot3',
                figure={
                    'data': [go.Choroplethmapbox(
                        z = [1,2],
                        geojson=counties,
                        locations=locations,
                        colorscale='Viridis',
                        colorbar_title="Precipitation"
                    )],
                    'layout': go.Layout(
                        mapbox_style="carto-positron",
                        mapbox_accesstoken=token,
                        mapbox_zoom=4,
                        mapbox_center = {"lat": 4.570868, "lon": -74.2973328}
                    )
                }
            )
        ])])

app.run_server(mode='inline')

SyntaxError: invalid syntax (<ipython-input-53-881cce0f089f>, line 1)